<h1>A1 Regression Model Development</h1>

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

## 1. Preparing Data
To start the assignment, I imported the packages I was going to use, as well as naming and reading the birthweight_low file.
After this, I stared analyzing the data and looked for any missing values. As there were some missing values, I did a histogram to decide whether to use the median, mode or mean. Following this step, I used the .fillna() funciton to fill the missing values with the median or mean. Finally I dropped the columns in the data of "omaps" and "fmaps" since this categories are used after the baby is born, and it's not an indicator that can help us predict the baby weight.

In [ ]:
# importing libraries
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.linear_model import ARDRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler

# setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


file = "./birthweight_low.xlsx"

# reading file in Pyhton
birthweight = pd.read_excel(io = file,
                           header = 0,
                           sheet_name = 0)

birthweight.head(n = 5)

In [ ]:
birthweight.shape

In [ ]:
# looking for missing values
birthweight.isnull().sum(axis = 0)

In [ ]:
# histogram to check features for the variables with missing values
sns.histplot(data   = birthweight,
             x      = 'meduc',
             kde    = True)

plt.title(label   = "Original Distribution of Mother's Education")
plt.xlabel(xlabel = "Meduc")
plt.ylabel(ylabel = "Count")

plt.show()

# histogram to check features for the variables with missing values
sns.histplot(data   = birthweight,
             x      = 'npvis',
             kde    = True)

plt.title(label   = "Original Distribution of Prenatal Visits")
plt.xlabel(xlabel = "Npvis")
plt.ylabel(ylabel = "Count")

plt.show()

# histogram to check features for the variables with missing values
sns.histplot(data   = birthweight,
             x      = 'feduc',
             kde    = True)

plt.title(label   = "Original Distribution of Father's Education")
plt.xlabel(xlabel = "Feduc")
plt.ylabel(ylabel = "Count")

plt.show()

In [ ]:
# filling the missing values with the mean and median
meduc_mean=birthweight['meduc'].min()
birthweight['meduc'].fillna(value=meduc_mean,
                                 inplace=True)

npvis_median=birthweight['npvis'].median()
birthweight['npvis'].fillna(value=2,
                                 inplace=True)

feduc_mean=birthweight['feduc'].mean()
birthweight['feduc'].fillna(value=12,
                                 inplace=True)
birthweight.isnull().any()

In [ ]:
# dropping variables that are not valuable for prediction
birthweight = birthweight.drop('omaps', axis = 1)
birthweight = birthweight.drop('fmaps', axis = 1)

## 2. Analyzing Data
After having the data correctly set, I started analyzing the y variable and checking for skewness to see if it is better to use log in each. 
Another way to analyze the data was doing boxplots for each of the variables and investigating the "normal" least birth weight a baby should have when being born. I placed a line in the boxplots to see which variables could be placed in categories, and started feature engineering.

In [ ]:
# developing a histogram to analyze the distribution of birthweight and checking for skewness
sns.histplot(data   = birthweight,
             x      = 'bwght',
             kde    = True)

plt.title(label   = "Original Distribution of Birthweight")
plt.xlabel(xlabel = "Birthweight") 
plt.ylabel(ylabel = "Count")

plt.show()

# log transforming bwght
birthweight['log_bwght'] = np.log(birthweight['bwght'])

# log transforming feduc
birthweight['log_feduc'] = np.log(birthweight['feduc'])

# developing a histogram
sns.histplot(data   = birthweight,
             x      = 'log_bwght',
             kde    = True)

plt.title(label   = "Logarithmic Distribution of Birthweight")
plt.xlabel(xlabel = "Birthweight") 
plt.ylabel(ylabel = "Count")

plt.show()

In [ ]:
# creating a loop to see the boxplots of each variable
columns = ['mage', 'meduc','monpre', 'npvis','fage','feduc','cigs','drink',
        'male', 'mwhte','mblck','moth','fwhte','fblck','foth']

for col in columns: 
    fig, ax = plt.subplots(figsize =(13, 8))
    sns.boxplot(x    = col,
                y    = 'bwght',
                data = birthweight)

    plt.title(label   = 'Boxplot with Interval Data')
    plt.xlabel(xlabel = col)
    plt.ylabel(ylabel = 'birth weight')
   
    # line on the x axis to see the minimum weight
    plt.axhline(y = 2500, color = "black", linestyle = '--')
    plt.show()

## 3. Feature Engineering
In the next lines of codes, I did categories based on the data, for example I divided the feature "drink" because we can see in the boxplot the average drink per day which can affect the weight below the line. I used the same reasoning for average cigarettes per day as well as for the father education.
I created a column to multiply the mother age with the average cigarettes and the average drink per day because this can directly affect the baby's weight. I started doing the OLS analysis to see which variables could affect more the R squared and the p values.

In [ ]:
# creating new categories for some of the expalantory variables
birthweight ['drink_mod'] = 0
birthweight['cigs_mod'] = 0
birthweight['log_feduc_mod'] = 0 

for index, col in birthweight.iterrows():
    if birthweight.loc[index,'drink']>=9:
        birthweight.loc[index,'drink_mod']=9
    elif birthweight.loc[index,'drink']<10:
        birthweight.loc[index,'drink_mod'] = birthweight.loc[index,'drink']
    else:
        print('error')
    if birthweight.loc[index,'cigs']>=21:
        birthweight.loc[index,'cigs_mod']=22
    elif birthweight.loc[index,'cigs']<21:
        birthweight.loc[index,'cigs_mod'] = birthweight.loc[index, 'cigs']
    else:
        print('error')
        
    if birthweight.loc[index,'log_feduc'] < 1:
        birthweight.loc[index,'log_feduc_mod']=2
    elif birthweight.loc[index,'log_feduc']> 1:
        birthweight.loc[index,'log_feduc_mod'] = birthweight.loc[index, 'log_feduc']
    else:
        print('error')

birthweight['mage:cigs']  = birthweight['mage'] * birthweight['cigs_mod']
birthweight['drink:mage'] = birthweight['drink'] * birthweight['drink_mod']
birthweight['cigs:fwhte'] = birthweight['cigs_mod'] * birthweight['fwhte']

In [ ]:
# log transforming variables
birthweight['log_monpre'] = np.log(birthweight['monpre'])
birthweight['log_npvis'] = np.log(birthweight['npvis'])
birthweight['log_fage'] = np.log(birthweight['fage'])


In [ ]:
# checking the correlation with bwght
birth_corr = birthweight.corr()
birth_corr.loc[['bwght','mage','meduc','monpre', 'npvis', 'fage', 'feduc','cigs',
                'drink','male', 'mwhte','mblck', 'fblck'] , 
               ['bwght','mage','meduc','monpre', 'npvis', 'fage', 'feduc','cigs',
                'drink','male', 'mwhte','mblck', 'fblck']].abs()


## 4. Linear Modeling

In [ ]:
# preparing explanatory variable 
birthweight_data  = birthweight.drop(['bwght','log_bwght'],axis = 1)


# preparing response variables
birthweight_target = birthweight.loc[ : , 'bwght']
log_birthweight_target = birthweight.loc[ : , 'log_bwght']

# preparing training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
            birthweight_data,
            birthweight_target,
            test_size = 0.25,
            random_state = 219)
print(f"""
Training Data
-------------
X-side: {x_train.shape}
y-side: {y_train.shape}


Testing Data
------------
X-side: {x_test.shape}
y-side: {y_test.shape}
""")

In [ ]:
# merging X_train and y_train for the OLS statsmodels

birthweight_train = pd.concat([x_train, y_train], axis = 1)

lm_best = smf.ols(formula = """ bwght ~ log_feduc_mod +
                                        drink_mod +                                       
                                        drink:mage+
                                        mage:cigs
                                                """,
                    data = birthweight_train)

results = lm_best.fit()
print(results.summary())


### 4.1 Ordinary Least Squares Regression

In [ ]:
# defining variables for x and y in the train and test sets
x = birthweight.loc[: , ['cigs_mod', 'drink','male','fage','mwhte','log_monpre']]
y = birthweight.loc[: , 'bwght']


#split into train and test set
x_train_OLS, x_test_OLS, y_train_OLS, y_test_OLS = train_test_split(x,y,test_size = 0.25, random_state =219)

#split into train and test set
x_train_full, x_test_full, y_train_full, y_test_full = train_test_split(birthweight_data,
                                        birthweight_target,test_size = 0.25, random_state =219)


# instantiating a model object
lr = LinearRegression()

# fitting to the training data
lr_fit = lr.fit(x_train_OLS, y_train_OLS)

#defining variables
lr_train_score = lr.score(x_train_OLS, y_train_OLS).round(4)
lr_test_score = lr.score(x_test_OLS, y_test_OLS).round(4)

# predicting on new data
lr_pred = lr_fit.predict(x_test_OLS)

# scoring the results
print('OLS Training Score :', lr.score(x_train_OLS, y_train_OLS).round(4)) 
print('OLS Testing Score  :',  lr.score(x_test_OLS, y_test).round(4)) 


# displaying the gap between training and testing
lr_test_gap = abs(lr_train_score - lr_test_score).round(4)

### 4.2 Lasso Regression Model

In [ ]:
# instantiating a model object
lasso_model = Lasso(alpha = 1.0,normalize = True) 


# fitting to the training data
lasso_fit = lasso_model.fit(x_train_OLS, y_train_OLS)


# predicting on new data
lasso_pred = lasso_fit.predict(x_test_OLS)


# scoring the results
print('Lasso Training Score :', lasso_model.score(x_train_OLS, y_train_OLS).round(4))
print('Lasso Testing Score  :', lasso_model.score(x_test_OLS, y_test_OLS).round(4))


# defining variables 
lasso_train_score = lasso_model.score(x_train_OLS, y_train_OLS).round(4) 
lasso_test_score  = lasso_model.score(x_test_OLS, y_test_OLS).round(4)   


# displaying the gap between training and testing
print('Lasso Train-Test Gap :', abs(lasso_train_score - lasso_test_score).round(4))
lasso_test_gap = abs(lasso_train_score - lasso_test_score).round(4)


In [ ]:
# zipping each feature name to its coefficient
lasso_model_values = zip(birthweight_data.columns, lasso_fit.coef_.round(decimals = 2))


# setting up a placeholder list to store model features
lasso_model_lst = [('intercept', lasso_fit.intercept_.round(decimals = 2))]


# loop for printing out each feature-coefficient pair 
for val in lasso_model_values:
    lasso_model_lst.append(val)
    
for pair in lasso_model_lst:
    print(pair)

### 4.3 Automatic Relevance Determination (ARD)

In [ ]:
# instantiating a model object
import sklearn.linear_model

ard_model = sklearn.linear_model.ARDRegression()

ard_fit = ard_model.fit(x_train_OLS, y_train_OLS)

# predicting on new data
ard_pred = ard_fit.predict(x_test_OLS)

print('Training Score:', ard_model.score(x_train_OLS, y_train_OLS).round(4))
print('Testing Score :',  ard_model.score(x_test_OLS, y_test_OLS).round(4))

# saving scoring data 
ard_train_score = ard_model.score(x_train_OLS, y_train_OLS).round(4)
ard_test_score  = ard_model.score(x_test_OLS, y_test_OLS).round(4)

# displaying the gap between training and testing
print('ARD Train-Test Gap :', abs(ard_train_score - ard_test_score).round(4))
ard_test_gap = abs(ard_train_score - ard_test_score).round(4)


### 4.4 K-Nearest Neighbors

In [ ]:

# instantiating a StandardScaler() object
scaler = StandardScaler()


# fitting the scaler with birthweight_data
scaler.fit(birthweight_data)


# transforming data 
x_scaled = scaler.transform(birthweight_data)

# converting scaled data into a DataFrame
x_scaled_df = pd.DataFrame(x_scaled)

# checking the results
x_scaled_df.describe().round(2)

# adding labels to the scaled DataFrame
x_scaled_df.columns = birthweight_data.columns

# splitting test and train
x_train, x_test, y_train, y_test = train_test_split(
            x_scaled,
            birthweight_target,
            test_size = 0.25,
            random_state = 219)

In [ ]:
# instantiating a KNN model object
knn = KNeighborsRegressor(n_neighbors = 11)

# fitting to the training data
knn_fit = knn.fit(x_train, y_train)

# predicting on new data
knn_pred = knn_fit.predict(x_test)

# scoring the results
print('KNN Training Score:', knn.score(x_train, y_train).round(4))
print('KNN Testing Score :',  knn.score(x_test, y_test).round(4))

# defining variables
knn_train_score = knn.score(x_train, y_train).round(4)
knn_test_score  = knn.score(x_test, y_test).round(4)

# displaying gap between training and testing
print('KNN Train-Test Gap:', abs(knn_train_score - knn_test_score).round(4))
knn_test_gap = abs(knn_train_score - knn_test_score).round(4)

In [ ]:
# creating lists for training set accuracy and test set accuracy
training_accuracy = []
test_accuracy     = []

# building a visualization of 1 to 50 neighbors
neighbors_settings = range(1, 51)


for n_neighbors in neighbors_settings:
    # Building the model
    clf = KNeighborsRegressor(n_neighbors = n_neighbors)
    clf.fit(x_train, y_train)
    
    # Recording the training set accuracy
    training_accuracy.append(clf.score(x_train, y_train))
    
    # Recording the generalization accuracy
    test_accuracy.append(clf.score(x_test, y_test))


# plotting the visualization
fig, ax = plt.subplots(figsize=(12,8))
plt.plot(neighbors_settings, training_accuracy, label = "training accuracy")
plt.plot(neighbors_settings, test_accuracy, label = "test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("n_neighbors")
plt.legend()
plt.show()

### 5. Final model 

In [ ]:
# comparing results of each model

print(f"""
Model             Train Score      Test Score      Train-Test Gap
-----             -----------      ----------      ----------
OLS                 {lr_train_score}           {lr_test_score}          {lr_test_gap}
Lasso[Final model]  {lasso_train_score}           {lasso_test_score}          {lasso_test_gap}
ARD                 {ard_train_score}           {ard_test_score}          {ard_test_gap}
KNN                 {knn_train_score}           {knn_test_score}          {knn_test_gap}
""")
